# 2024/06/05 LODES User Manual
_Author: Meaghan Freund_

Queries and uses of the LODES ADRIO Template

Set up for the LODES call, users are able to change the call depending on which states or further granularities they specifically want to  see

**CHANGE FOR GEOSCOPES PENDING**

In [ ]:
from epymorph.data_shape import Shapes
from epymorph.data_type import CentroidDType
from epymorph.geo.adrio.census.adrio_census import CensusGeography, Granularity
from epymorph.geo.spec import DynamicGeoSpec, Year, attrib

spec = DynamicGeoSpec(
    attributes=[
        attrib('label', dtype=str, shape=Shapes.N),
        attrib('population', dtype=int, shape=Shapes.N),
        attrib('population_by_age', dtype=int, shape=Shapes.NxA(3)),
        attrib('centroid', dtype=CentroidDType, shape=Shapes.N),
        attrib('geoid', dtype=str, shape=Shapes.N),
        attrib('dissimilarity_index', dtype=float, shape=Shapes.N),
        attrib('median_income', dtype=int, shape=Shapes.N),
        attrib('pop_density_km2', dtype=float, shape=Shapes.N),
        attrib('commuters', dtype=int, shape=Shapes.NxN),
    ],
    time_period=Year(2015),
    geography=CensusGeography(granularity=Granularity.COUNTY, filter={
        'state': ['04', '08', '49', '35', '32'],
        'county': ['*'],
        'tract': ['*'],
        'block group': ['*']
    }),
    source={
        'label': 'Census:name',
        'population': 'Census',
        'population_by_age': 'Census',
        'centroid': 'Census',
        'geoid': 'Census',
        'dissimilarity_index': 'Census',
        'median_income': 'Census',
        'pop_density_km2': 'Census',
        'commuters': 'Census',
    }
)

with open('./epymorph/data/geo/us_sw_counties_2015.geo', mode='w', encoding='utf-8') as f:
    json = spec.serialize()
    f.write(json)

In [ ]:
from epymorph.geo.adrio import adrio_maker_library
from epymorph.geo.dynamic import DynamicGeo

geo = DynamicGeo.from_library(spec, adrio_maker_library)

# Query Types

The first and most general type of query 